In [156]:
# First import necessary libraries
import pandas as pd
from sklearn.ensemble import StackingRegressor
import numpy as np

# Decision trees
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler

# Linear model as meta-learn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

# Create generic dataset for regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [157]:
df= pd.read_csv("Bruska_data.csv")
df.head()

,time stamp,Bruska_wind speed,Bruska_wind direction,Bruska_air density,Zelengrad_wind speed,Zelengrad_wind direction,Zelengrad_air density,Bruska_active power output,Bruska_normally operating number
0,2018-01-01 00:00:00,9.363334,172.79999,1.210,10.635238,165.28897,1.207,21.608133,14.0
1,2018-01-01 01:00:00,9.266666,174.99990,1.210,10.261664,166.09566,1.207,21.050860,14.0
2,2018-01-01 02:00:00,9.586667,174.19984,1.209,13.286666,161.92586,1.207,21.257930,14.0
3,2018-01-01 03:00:00,9.320000,167.53835,1.208,14.777143,159.17044,1.206,20.700977,14.0
4,2018-01-01 04:00:00,9.267742,160.91630,1.207,11.447466,151.64941,1.205,20.331597,14.0


In [158]:
df.shape

(17352, 9)

In [159]:
df.dropna(inplace=True)

In [160]:
df.drop(["time stamp"], axis= 1, inplace= True)

In [161]:
df.drop(["Bruska_normally operating number"], axis= 1, inplace= True)

In [162]:
df.describe()

,Bruska_wind speed,Bruska_wind direction,Bruska_air density,Zelengrad_wind speed,Zelengrad_wind direction,Zelengrad_air density,Bruska_active power output
count,17189.000000,17189.000000,17189.000000,17189.000000,17189.000000,17189.000000,17189.000000
mean,7.406829,124.526991,1.175567,7.484516,133.140676,1.173686,12.986623
std,4.659973,106.650245,0.035333,5.126521,92.257722,0.036053,12.696268
min,0.503352,0.000000,1.093001,0.572682,0.420131,1.091000,0.000000
25%,3.754839,32.928550,1.147000,3.405714,45.913185,1.145000,1.231610
50%,6.476667,87.799580,1.174000,6.075128,111.267845,1.172000,8.279612
75%,10.194118,201.179170,1.200000,10.486428,208.017650,1.198000,24.654873
max,27.918930,359.945370,1.291999,30.919079,359.072630,1.290000,35.967899


In [163]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)

In [164]:
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['Bruska_active power output']])

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42,shuffle=False)

In [166]:
def get_stacking(input_shape=None):
    level0 = list()
    level0.append(('cat', CatBoostRegressor(learning_rate=0.3,max_depth=2,l2_leaf_reg=0.2,iterations=1000)))
    level0.append(('xg', XGBRegressor(tree_method='approx',n_estimators=300,max_depth=9,learning_rate=0.1,colsample_bytree=1)))
    level0.append(('rf', RandomForestRegressor(n_estimators=120,min_samples_split=2,min_samples_leaf=2,max_samples=0.5,max_features=1,max_depth=8,bootstrap=True)))
    level1 = LinearRegression()
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=2, verbose=1)
    return model

In [167]:
import time
start= time.time()
model = get_stacking()
model.fit(X_train, y_train)
end=time.time()
continental= end-start
print("continential",continental)

c:\Users\sunrise\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_stacking.py:865: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0:	learn: 0.2591969	total: 4.45ms	remaining: 4.45s
1:	learn: 0.1909529	total: 7.63ms	remaining: 3.81s
2:	learn: 0.1405046	total: 12.7ms	remaining: 4.21s
3:	learn: 0.1050689	total: 18.8ms	remaining: 4.69s
4:	learn: 0.0792801	total: 27.8ms	remaining: 5.54s
5:	learn: 0.0625684	total: 31.2ms	remaining: 5.16s
6:	learn: 0.0486842	total: 44ms	remaining: 6.24s
7:	learn: 0.0392170	total: 47.3ms	remaining: 5.86s
8:	learn: 0.0328966	total: 50.8ms	remaining: 5.6s
9:	learn: 0.0293823	total: 58.9ms	remaining: 5.83s
10:	learn: 0.0263694	total: 63ms	remaining: 5.66s
11:	learn: 0.0249192	total: 66.3ms	remaining: 5.46s
12:	learn: 0.0238612	total: 71.2ms	remaining: 5.41s
13:	learn: 0.0232934	total: 79.5ms	remaining: 5.6s
14:	learn: 0.0227567	total: 83ms	remaining: 5.45s
15:	learn: 0.0217880	total: 87.1ms	remaining: 5.36s
16:	learn: 0.0215537	total: 92.6ms	remaining: 5.35s
17:	learn: 0.0209642	total: 96.4ms	remaining: 5.26s
18:	learn: 0.0208063	total: 99.9ms	remaining: 5.16s
19:	learn: 0.0203436	total: 10

37:	learn: 0.0142488	total: 462ms	remaining: 11.7s
38:	learn: 0.0142160	total: 468ms	remaining: 11.5s
39:	learn: 0.0138395	total: 473ms	remaining: 11.4s
40:	learn: 0.0135101	total: 478ms	remaining: 11.2s
41:	learn: 0.0134337	total: 485ms	remaining: 11.1s
42:	learn: 0.0131717	total: 491ms	remaining: 10.9s
43:	learn: 0.0129254	total: 520ms	remaining: 11.3s
44:	learn: 0.0127607	total: 526ms	remaining: 11.2s
45:	learn: 0.0125949	total: 535ms	remaining: 11.1s
46:	learn: 0.0124269	total: 549ms	remaining: 11.1s
47:	learn: 0.0122057	total: 555ms	remaining: 11s
48:	learn: 0.0119701	total: 584ms	remaining: 11.3s
49:	learn: 0.0118133	total: 615ms	remaining: 11.7s
50:	learn: 0.0116147	total: 618ms	remaining: 11.5s
51:	learn: 0.0115044	total: 622ms	remaining: 11.3s
52:	learn: 0.0113220	total: 627ms	remaining: 11.2s
53:	learn: 0.0111325	total: 630ms	remaining: 11s
54:	learn: 0.0110846	total: 633ms	remaining: 10.9s
55:	learn: 0.0109550	total: 637ms	remaining: 10.7s
56:	learn: 0.0107993	total: 647ms	r

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.2640389	total: 2.43ms	remaining: 2.42s
1:	learn: 0.1936280	total: 4.11ms	remaining: 2.05s
2:	learn: 0.1441773	total: 5.48ms	remaining: 1.82s
3:	learn: 0.1074170	total: 6.85ms	remaining: 1.71s
4:	learn: 0.0819192	total: 8.25ms	remaining: 1.64s
5:	learn: 0.0627227	total: 9.63ms	remaining: 1.59s
6:	learn: 0.0491420	total: 11ms	remaining: 1.56s
7:	learn: 0.0399687	total: 12.4ms	remaining: 1.54s
8:	learn: 0.0345621	total: 14.4ms	remaining: 1.59s
9:	learn: 0.0305401	total: 16.1ms	remaining: 1.59s
10:	learn: 0.0279228	total: 17.5ms	remaining: 1.57s
11:	learn: 0.0254752	total: 18.9ms	remaining: 1.56s
12:	learn: 0.0241656	total: 20.3ms	remaining: 1.54s
13:	learn: 0.0234161	total: 21.7ms	remaining: 1.53s
14:	learn: 0.0228240	total: 23.1ms	remaining: 1.52s
15:	learn: 0.0218627	total: 24.6ms	remaining: 1.51s
16:	learn: 0.0214951	total: 26ms	remaining: 1.5s
17:	learn: 0.0211284	total: 27.4ms	remaining: 1.5s
18:	learn: 0.0204979	total: 30.7ms	remaining: 1.58s
19:	learn: 0.0203141	total: 

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


continential 18.18976593017578


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s finished


In [168]:
import time
start= time.time()
prediction=model.predict(X_test)
end=time.time()
continental=end-start
print("inference time:",continental)

inference time: 0.17943024635314941


In [169]:
prediction

array([1.45091788e-03, 3.61266129e-06, 2.47084265e-05, ...,
       3.06549185e-02, 4.91810414e-02, 5.06487106e-03])

In [170]:
prediction.shape

(3438,)

In [171]:
Prediction = prediction.reshape(prediction.shape[0], 1)

In [172]:
Pred= s2.inverse_transform(Prediction)

In [173]:
y_test

array([[0.00178585],
       [0.        ],
       [0.        ],
       ...,
       [0.03007314],
       [0.04934149],
       [0.00554754]])

In [174]:
y_test.shape

(3438, 1)

In [175]:
Actual= s2.inverse_transform(y_test)

In [176]:
z=Actual
x=Pred

In [177]:
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE
from sklearn.metrics import mean_absolute_percentage_error
MAPE=mean_absolute_percentage_error(z,x)
MAPE
print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)
print("Mean Absolute Percentage Error:",MAPE)

Root Mean Square Error: 0.036971547916981484
Mean Square Error: 0.0013668953553776578
Mean Absolute Error: 0.024386082705566164
Mean Absolute Percentage Error: 1388690940904.9675


In [178]:
max= x.max()
max

35.850913767192246

In [179]:
min=x.min()
min

-0.030801611886678425

In [180]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 0.10303729218736847


In [181]:
Calculated = pd.DataFrame(Pred, columns = ['Prediction'])

In [182]:
Calculated.head()

,Prediction
0,0.052186
1,0.000130
2,0.000889
3,0.001468
4,0.000699


In [183]:
Actual = pd.DataFrame(Actual, columns = ['Actual'])

In [184]:
Actual.head()

,Actual
0,0.064233
1,0.000000
2,0.000000
3,0.000000
4,0.000000


In [185]:
combined_df = pd.concat([Calculated, Actual], axis=1)

In [186]:
combined_df.to_csv(r'C:\Users\sunrise\Desktop\Wind speed forecasting\Bruska\stacking_model\Hybrid_prediction.csv', index = False)